# Predicting Airbnb Prices for Munich

The goal of our data mining project is to predict prices for new Airbnb listings in Munich. To achieve this, we will train a regression model on existing Airbnb data from www.insideairbnb.com.

## Table of Contents
##### [1 Preprocessing](#preprocessing)
##### [2 Data Mining](#data_mining)
##### [3 Interpretation and Evaluation](#interpretation_evaluation)

<a id='preprocessing'></a>
## 1 Preprocessing

In [2]:
%run modules/preprocessing.py
preprocessed_df = load_and_preprocess_dataset()

2019-11-23 13:56:10 : Dataset loaded and preprocessed.


In [7]:
%run modules/preprocessing.py
features = select_best_features(preprocessed_df, number_of_features = 70) # Total number of features: 50
label = preprocessed_df['maximum_price']

Selected Features:  15                                       maximum_nights
12                                     security_deposit
13                                         cleaning_fee
1                             host_total_listings_count
10                                                 beds
7                                          accommodates
9                                              bedrooms
17                                  cancellation_policy
6                                             room_type
28                                     24-hour check-in
68                                      distance_centre
14                                       minimum_nights
55                                        tudering-riem
43                         ludwigsvorstadt-isarvorstadt
63                                       altstadt-lehel
19                     require_guest_phone_verification
18                        require_guest_profile_picture
48                          

In [12]:
#
bins     = [0,100,200,300,400,500,600,700,800,900,1000,2000,5001]
y_binned = np.digitize(label, bins)

count    10136.000000
mean       129.881314
std        141.040814
min          8.000000
25%         60.000000
50%         94.000000
75%        150.000000
max       5000.000000
Name: maximum_price, dtype: float64


In [ ]:
# if you want to delete outliers, execute this after the train-test-split:

# %run modules/preprocessing.py
# x_train, y_train = delete_price_outliers(x_train, y_train)

<a id='data_mining'></a>
## 2 Data Mining

### 2.1 Evaluation of a Dummy Regressor

In [3]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor
from math import sqrt

# k-fold cross-validation (k = 10)
scores = []
rmse = []
k_fold_cross_validation = KFold(10, True, 1)
for train_index, test_index in k_fold_cross_validation.split(features):

    # Split the dataset for training and testing
    x_train, x_test, y_train, y_test = features.loc[train_index, :], features.loc[test_index, :], label[train_index], label[test_index]
    
    # Dummy Regressor
    regressor = DummyRegressor(strategy='median')
    regressor.fit(x_train, y_train)

    # Evaluation using testing dataset
    scores.append(regressor.score(x_test, y_test))  
    predictions = regressor.predict(x_test)
    rmse.append(sqrt(mean_squared_error(y_test, predictions)))

# Calculate performance measures
print("Dummy Regressor: ", str(np.mean(scores)))
print("RMSE: ", str(np.mean(rmse)))

Dummy Regressor:  -0.07154551044155429
RMSE:  143.85943496805479


### 2.2 Evaluation of different Regression Approaches

In [16]:
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import itertools
from math import sqrt

x_train, x_test, y_train, y_test = train_test_split(features, label, test_size = 0.2, random_state = 0)

x_train, y_train = delete_price_outliers(x_train, y_train)

# test different regression approaches
estimators = [ LinearRegression(), Ridge(), KNeighborsRegressor(), DecisionTreeRegressor(), MLPRegressor(), SVR() ]
svr = [ SVR() ]
pipeline = Pipeline( [ ('preprocessing', StandardScaler()), ('estimator', None) ])

# define a parameter grid
parameters = {
    'estimator': estimators
}

# define and run a grid search using MSE as scoring metric
search = GridSearchCV(pipeline, parameters, cv=10, scoring='neg_mean_squared_error')
search.fit(x_train, y_train)

# evaluate on test set
predictions = search.predict(x_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("Best Model: {}".format(search.best_params_))
print("RMSE: {}".format(sqrt(mse)))
print("R^2: {}".format(r2))

Best Model: {'estimator': MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=200, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)}
RMSE: 142.5537543342086
R^2: 0.2519363931631057


### 2.3 Evaluation of the Support Vector Machine

In [ ]:
%run modules/evaluation.py

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

best_r2 = 0

# k-fold cross-validation (k = 10)
scores = []
k_fold_cross_validation = KFold(10, True, 1)
for train_index, test_index in k_fold_cross_validation.split(features):

    # Split the dataset for training and testing
    x_train, x_test, y_train, y_test = features.loc[train_index, :], features.loc[test_index, :], label[train_index], label[test_index]

    # Support Vector Regressor (SVR) using training dataset
    svr = SVR(kernel='linear', C = 0.7)
    svr.fit(x_train, y_train)

    # Evaluation using testing dataset
    scores.append(svr.score(x_test, y_test))  

# Calculate performance measures
print("r2: ", str(np.mean(scores)))

In [33]:
%run modules/evaluation.py

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

best_r2 = 0

# Generate all feature combinations
feature_combinations = generate_feature_combinations(already_preprocessed)

for feature_combination in feature_combinations:
    
    # Filter the selected features
    selected_features = already_preprocessed[feature_combination]
    
    # k-fold cross-validation (k = 10)
    scores = []
    k_fold_cross_validation = KFold(10, True, 1)
    for train_index, test_index in k_fold_cross_validation.split(selected_features):
    
        # Split the dataset for training and testing
        x_train, x_test, y_train, y_test = selected_features.loc[train_index, :], selected_features.loc[test_index, :], label[train_index], label[test_index]

        # Support Vector Regressor (SVR) using training dataset
        svr = SVR(kernel='linear', C = 0.7)
        svr.fit(x_train, y_train)

        # Evaluation using testing dataset
        scores.append(svr.score(x_test, y_test))  

    # Calculate performance measures
    print(np.mean(scores), " - ", feature_combination)

    # Save best model
    if(np.mean(scores) > best_r2):
          best_r2 = np.mean(scores)
        
print("Best: ", best_r2)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

### 2.4 Evaluation of Linear Regression

In [127]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features, label, test_size = 0.2, stratify=y_binned, random_state = 42)

x_train, y_train = delete_price_outliers(x_train, y_train)

reg = LinearRegression()
reg.fit(x_train, y_train)
prediction = reg.predict(x_test)
mse = mean_squared_error(y_test, prediction)
r2 = r2_score(y_test, prediction)
print("MSE:", mse)
print("RMSE:", sqrt(mse))
print("R^2:", r2)

MSE: 11276.312930767506
RMSE: 106.18998507753689
R^2: 0.3832052657898344


In [53]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features, label, test_size = 0.2, random_state = 42)

#x_train, y_train = delete_price_outliers(x_train, y_train)

reg = LinearRegression()
reg.fit(x_train, y_train)
prediction = reg.predict(x_test)
mse = mean_squared_error(y_test, prediction)
r2 = r2_score(y_test, prediction)
print("MSE:", mse)
print("RMSE:", sqrt(mse))
print("R^2:", r2)

MSE: 9026.717425519482
RMSE: 95.00903865169609
R^2: 0.42516874391327064


2.5 Evaluation of Advanced Regression

In [128]:
#1. Ridge Regression
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

rr_est = Ridge()
param = {"alpha": [1e-10, 1e-8, 1e-4,1e-3, 1e-2, 1, 5, 10, 20]}

rr_est_opt = GridSearchCV(rr_est, param, scoring="neg_mean_squared_error", cv=5)
rr_est_opt.fit(x_train, y_train)

# Get best param
print(rr_est_opt.best_params_)
print(rr_est_opt.best_score_)

{'alpha': 20}
-4696.814613726855


In [129]:
# Test with alpha 20
from sklearn.linear_model import ridge
rr = Ridge(alpha = 20)
rr.fit(x_train, y_train)
rr.coef_

#Test and Evaluate
rr_predictions = rr.predict(x_test)

mse = mean_squared_error(y_test, rr_predictions)
r2 = r2_score(y_test, rr_predictions)

print("MSE:", mse)
print("RMSE:", sqrt(mse))
print("R^2:", r2)

MSE: 11276.885237588422
RMSE: 106.19267977402407
R^2: 0.3831739616006318


<a id='interpretation_evaluation'></a>
## 3 Interpretation and Evaluation